### Load data from .odt format

In [1]:
import pandas as pd
from odf.opendocument import load
from odf import text, teletype

def load_data(filepath):
    raw_data = []
    textdoc = load(filepath)
    allparas = textdoc.getElementsByType(text.P)
    for line in allparas:
        raw_data.append(teletype.extractText(line))
    return raw_data

In [2]:
raw_data = load_data('dataset/data.odt')
while '' in raw_data:
    raw_data.remove('')
raw_data

['Wypowiedzi OG',
 'Przywitanie',
 '- Dzień dobry.',
 '[',
 'dzień dobry',
 'witać',
 ']',
 'Wyjaśnienie powodu wizyty',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego.',
 '[',
 'ja wniosek dowód1 mieć',
 'ja wniosek dowód2 mieć',
 ']',
 '- Czy mogę odebrać dowód?',
 '[',
 'czy już nowy dowód1',
 'czy już nowy dowód2',
 'być mój nowy dowód1',
 'być mój nowy dowód2',
 ']',
 '- Chcę zgłosić utratę dowodu osobistego.',
 '[',
 'ja dowód1 zgubić',
 'ja dowód2 zgubić',
 ']',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego dla mojego dziecka.',
 '[',
 'ja dowód1 moje dziecko chcieć',
 'ja dowód2 moje dziecko chcieć',
 ']',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego dla mojej żony.',
 '[',
 'ja dowód1 moja żona chcieć',
 'ja dowód2 moja żona chcieć',
 ']',
 'Obsługa',
 '- Zmieniłem adres.',
 '[',
 'dom przenieść się',
 'mieszkanie zmiana1',
 'mieszkanie zmiana2',
 'adres1 zmiana1',
 'adres2 zmiana1',
 'adres1 zmiana2',
 'adres2 zmiana2',
 ']',
 '- Upłynął termin ważności dowo

In [18]:
def split_data_from_list(raw_data):
    pl_sentence = []
    sentence = []
    i=0
    while i < len(raw_data):
        if raw_data[i+1] == '[':
            value = raw_data[i]
            i += 2
            while i < len(raw_data):
                if raw_data[i] == ']':
                    break
                pl_sentence.append(value[1:])
                sentence.append(raw_data[i])
                i += 1
        i += 1
    return pl_sentence, sentence

In [19]:
s1, s2 = split_data_from_list(raw_data)

In [23]:
data = pd.DataFrame({'pl':s1, 'mig':s2})
print(data[0:10])

0                                         Dzień dobry.
1                                         Dzień dobry.
2     Chcę złożyć wniosek o wydanie dowodu osobistego.
3     Chcę złożyć wniosek o wydanie dowodu osobistego.
4                              Czy mogę odebrać dowód?
5                              Czy mogę odebrać dowód?
6                              Czy mogę odebrać dowód?
7                              Czy mogę odebrać dowód?
8               Chcę zgłosić utratę dowodu osobistego.
9               Chcę zgłosić utratę dowodu osobistego.
Name: pl, dtype: object


### Creating a dataset

In [25]:
from datasets import Dataset, DatasetDict

raw_dataset_list = []
for i in range(0, len(data)):
    raw_dataset_list.append({'translation' : {'pl' : data['pl'][i], 'mig' : data['mig'][i]}})
    
raw_dataset = Dataset.from_list(raw_dataset_list)
raw_dataset

Dataset({
    features: ['translation'],
    num_rows: 123
})

### Split data into train, validation and test dataset

In [26]:
train_testvalid = raw_dataset.train_test_split(test_size=0.3)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 86
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 19
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 18
    })
})